In [1]:

pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [4]:

pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [6]:

# Step 1: Import necessary libraries
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import classification_report, accuracy_score
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

# Step 2: Fetch and prepare the dataset
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets

# Combine X and y for exploration
df = pd.concat([X, y], axis=1)

# Handle missing values
df.fillna(df.mean(), inplace=True)

# Separate features and target
X_clean = df.drop(columns=['num'])  # Assuming 'num' is the target column
y_clean = df['num']

# Step 3: Feature Engineering
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Polynomial Features for interactions
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Feature Selection using Random Forest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
X_selected = selector.fit_transform(X_poly, y_clean)

# Step 4: Handle Imbalance using SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_selected, y_clean)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 5: Define and Tune Models

# Random Forest Tuning
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_tuned_model = RandomizedSearchCV(rf_model, rf_params, n_iter=30, cv=5, scoring='accuracy', random_state=42)
rf_tuned_model.fit(X_train, y_train)

# Evaluate Random Forest
rf_best_model = rf_tuned_model.best_estimator_
y_pred_rf = rf_best_model.predict(X_test)
print("Random Forest")
print(classification_report(y_test, y_pred_rf))

# SVM Tuning
svm_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf']
}
svm_model = SVC(class_weight='balanced', probability=True, random_state=42)
svm_tuned_model = RandomizedSearchCV(svm_model, svm_params, n_iter=30, cv=5, scoring='accuracy', random_state=42)
svm_tuned_model.fit(X_train, y_train)

# Evaluate SVM
svm_best_model = svm_tuned_model.best_estimator_
y_pred_svm = svm_best_model.predict(X_test)
print("SVM")
print(classification_report(y_test, y_pred_svm))

# XGBoost Tuning
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_tuned_model = RandomizedSearchCV(xgb_model, xgb_params, n_iter=30, cv=5, scoring='accuracy', random_state=42)
xgb_tuned_model.fit(X_train, y_train)

# Evaluate XGBoost
xgb_best_model = xgb_tuned_model.best_estimator_
y_pred_xgb = xgb_best_model.predict(X_test)
print("XGBoost")
print(classification_report(y_test, y_pred_xgb))

# Logistic Regression
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000)
lr_model.fit(X_train, y_train)

# Evaluate Logistic Regression
y_pred_lr = lr_model.predict(X_test)
print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))

Random Forest
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        17
           1       0.93      0.89      0.91        28
           2       1.00      0.94      0.97        32
           3       0.91      1.00      0.95        20
           4       1.00      1.00      1.00        34

    accuracy                           0.95       131
   macro avg       0.94      0.95      0.95       131
weighted avg       0.96      0.95      0.95       131

SVM
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        17
           1       0.96      0.96      0.96        28
           2       0.97      1.00      0.98        32
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        34

    accuracy                           0.98       131
   macro avg       0.97      0.97      0.97       131
weighted avg       0.98      0.98      0.98       131

XGBo

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier



# K-Nearest Neighbors (KNN)
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
knn_model = KNeighborsClassifier()
knn_tuned_model = RandomizedSearchCV(knn_model, knn_params, n_iter=30, cv=5, scoring='accuracy', random_state=42)
knn_tuned_model.fit(X_train, y_train)

# Evaluate KNN
knn_best_model = knn_tuned_model.best_estimator_
y_pred_knn = knn_best_model.predict(X_test)
print("K-Nearest Neighbors")
print(classification_report(y_test, y_pred_knn))


K-Nearest Neighbors
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        17
           1       0.90      1.00      0.95        28
           2       1.00      1.00      1.00        32
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        34

    accuracy                           0.98       131
   macro avg       0.98      0.96      0.97       131
weighted avg       0.98      0.98      0.98       131



In [14]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
from sklearn.decomposition import PCA

# Enhanced Preprocessing
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Adding Polynomial Features for interactions (degree = 3 for more interaction depth)
poly = PolynomialFeatures(degree=3, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Use PCA to reduce dimensionality (keeping 95% variance)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_poly)

# Feature Selection using Random Forest to reduce noise and redundant features
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
X_selected = selector.fit_transform(X_pca, y_clean)

# Handle Imbalance using SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_selected, y_clean)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Naive Bayes with Enhanced Preprocessing
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Evaluate Naive Bayes
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes with Enhanced Preprocessing")
print(classification_report(y_test, y_pred_nb))

# Decision Tree with Hyperparameter Tuning
dt_params = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}
dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=42)
dt_tuned_model = RandomizedSearchCV(dt_model, dt_params, n_iter=50, cv=5, scoring='accuracy', random_state=42)
dt_tuned_model.fit(X_train, y_train)

# Evaluate Decision Tree
dt_best_model = dt_tuned_model.best_estimator_
y_pred_dt = dt_best_model.predict(X_test)
print("Decision Tree with Enhanced Preprocessing")
print(classification_report(y_test, y_pred_dt))


Naive Bayes with Enhanced Preprocessing
              precision    recall  f1-score   support

           0       0.58      0.58      0.58        12
           1       0.43      0.86      0.57        28
           2       0.89      0.27      0.41        30
           3       0.78      0.56      0.65        25
           4       0.80      0.80      0.80        30

    accuracy                           0.62       125
   macro avg       0.70      0.61      0.60       125
weighted avg       0.71      0.62      0.60       125

Decision Tree with Enhanced Preprocessing
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.81      0.61      0.69        28
           2       0.77      0.80      0.79        30
           3       0.79      0.76      0.78        25
           4       0.72      0.93      0.81        30

    accuracy                           0.77       125
   macro avg       0.78      0.75      0.76      

In [28]:
# Step 1: Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import classification_report, accuracy_score
from imblearn.combine import SMOTEENN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Step 2: Fetch and prepare the dataset
heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
y = heart_disease.data.targets

# Combine X and y for exploration
df = pd.concat([X, y], axis=1)

# Handle missing values
df.fillna(df.mean(), inplace=True)

# Separate features and target
X_clean = df.drop(columns=['num'])  # Assuming 'num' is the target column
y_clean = df['num']

# Step 3: Feature Engineering
# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Polynomial Features for interactions
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Step 4: Handle Imbalance using SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smoteenn.fit_resample(X_poly, y_clean)

# Reshape data for CNN (as CNN expects a 3D shape: [samples, time steps, features])
X_resampled_reshaped = X_resampled.reshape(X_resampled.shape[0], X_resampled.shape[1], 1)

# Convert target to categorical (for multiclass classification)
y_resampled_categorical = to_categorical(y_resampled)

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled_reshaped, y_resampled_categorical, test_size=0.2, random_state=42)

# Step 6: Define CNN Model
cnn_model = Sequential()

# 1D Convolution Layer
cnn_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn_model.add(BatchNormalization())  # Batch normalization for faster convergence

# Max Pooling Layer
cnn_model.add(MaxPooling1D(pool_size=2))

# Additional Conv Layer to increase capacity
cnn_model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling1D(pool_size=2))

# Flatten layer
cnn_model.add(Flatten())

# Fully Connected Dense Layers
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))  # Regularization
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(BatchNormalization())

# Output Layer (for multiclass classification)
cnn_model.add(Dense(y_resampled_categorical.shape[1], activation='softmax'))

# Compile the model
cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 7: Set callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Step 8: Train the CNN Model
history = cnn_model.fit(X_train, y_train, 
                        epochs=100, 
                        validation_split=0.2, 
                        batch_size=32, 
                        callbacks=[early_stopping, reduce_lr],
                        verbose=2)

# Step 9: Evaluate the CNN Model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)

# Predict and calculate classification report
y_pred_cnn = cnn_model.predict(X_test)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Step 10: Display classification report
print(f"CNN Model Test Accuracy: {test_accuracy * 100:.2f}%")
print("Classification Report")
print(classification_report(y_test_classes, y_pred_cnn_classes))



Epoch 1/100
14/14 - 1s - 59ms/step - accuracy: 0.3531 - loss: 1.7812 - val_accuracy: 0.4811 - val_loss: 1.4764 - learning_rate: 0.0010
Epoch 2/100
14/14 - 0s - 6ms/step - accuracy: 0.5379 - loss: 1.1883 - val_accuracy: 0.5283 - val_loss: 1.3977 - learning_rate: 0.0010
Epoch 3/100
14/14 - 0s - 6ms/step - accuracy: 0.6469 - loss: 0.8700 - val_accuracy: 0.5755 - val_loss: 1.3560 - learning_rate: 0.0010
Epoch 4/100
14/14 - 0s - 6ms/step - accuracy: 0.7607 - loss: 0.7256 - val_accuracy: 0.5094 - val_loss: 1.3193 - learning_rate: 0.0010
Epoch 5/100
14/14 - 0s - 6ms/step - accuracy: 0.7915 - loss: 0.5685 - val_accuracy: 0.5000 - val_loss: 1.2960 - learning_rate: 0.0010
Epoch 6/100
14/14 - 0s - 6ms/step - accuracy: 0.8341 - loss: 0.5002 - val_accuracy: 0.5189 - val_loss: 1.2588 - learning_rate: 0.0010
Epoch 7/100
14/14 - 0s - 6ms/step - accuracy: 0.8412 - loss: 0.4828 - val_accuracy: 0.5472 - val_loss: 1.2034 - learning_rate: 0.0010
Epoch 8/100
14/14 - 0s - 6ms/step - accuracy: 0.8697 - loss: 

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# Step 2: Define the Transformer Model with Dynamic Positional Encoding for Tabular Data
class TabularTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=2, dim_feedforward=128):
        super(TabularTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        
        # Define Transformer Encoder Layer with batch_first=True for improved performance
        transformer_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        
        # Reshape to add sequence dimension, making it compatible with Transformer
        x = x.unsqueeze(1)  # Shape becomes (batch_size, 1, d_model)
        
        # Dynamic positional encoding
        pos_encoder = torch.zeros(x.size()).to(x.device)
        x = x + pos_encoder  # Add positional encoding

        x = self.transformer(x)  # Apply transformer
        x = x.mean(dim=1)  # Pooling across the "sequence" dimension
        return self.fc(x)

# Instantiate and test model
model = TabularTransformer(input_dim=input_dim, num_classes=num_classes)


# Step 3: Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training Loop
epochs = 50
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Step 5: Evaluation
model.eval()
y_pred = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Transformer Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 10/50, Loss: 0.0212
Epoch 20/50, Loss: 0.0045
Epoch 30/50, Loss: 0.0069
Epoch 40/50, Loss: 0.0026
Epoch 50/50, Loss: 0.0006
Transformer Test Accuracy: 95.45%


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)
# Updated Transformer Model for Improved Performance
class EnhancedTabularTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=8, num_layers=4, dim_feedforward=256, dropout=0.3):
        super(EnhancedTabularTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        
        # Updated Transformer Encoder with additional layers and dropout for regularization
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add sequence dimension for Transformer compatibility
        
        pos_encoder = torch.zeros(x.size()).to(x.device)  # Dynamic positional encoding
        x = x + pos_encoder  # Apply positional encoding

        x = self.transformer(x)  # Transformer processing
        x = x.mean(dim=1)  # Pooling
        return self.fc(x)

# Instantiate model with increased dimensions
model = EnhancedTabularTransformer(input_dim=input_dim, num_classes=num_classes)

# Redefine the optimizer with a lower learning rate for finer tuning
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Increased epochs for training
epochs = 100
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate the updated model
model.eval()
y_pred = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Enhanced Transformer Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 10/100, Loss: 0.0146
Epoch 20/100, Loss: 0.0064
Epoch 30/100, Loss: 0.0038
Epoch 40/100, Loss: 0.0018
Epoch 50/100, Loss: 0.0013
Epoch 60/100, Loss: 0.0016
Epoch 70/100, Loss: 0.0028
Epoch 80/100, Loss: 0.1013
Epoch 90/100, Loss: 0.0010
Epoch 100/100, Loss: 0.0010
Enhanced Transformer Test Accuracy: 98.48%


In [26]:
from torch.optim.lr_scheduler import CosineAnnealingLR

# Define optimizer with a scheduler and early stopping patience
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# Early stopping parameters
best_accuracy = 0
patience, trigger_times = 10, 0

for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    # Update learning rate
    scheduler.step()

    # Validation accuracy check for early stopping
    model.eval()
    y_val_pred = []
    with torch.no_grad():
        for X_batch, _ in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_val_pred.extend(predicted.numpy())
    val_accuracy = accuracy_score(y_test, y_val_pred)

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        trigger_times = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping!")
            break
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy * 100:.2f}%")

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
print(f"Best Validation Accuracy Achieved: {best_accuracy * 100:.2f}%")


Epoch 10/100, Validation Accuracy: 97.73%
Early stopping!
Best Validation Accuracy Achieved: 98.48%


/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_75630/2839455758.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer

# Initialize tokenizer for clinical text processing
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define CNN for ECG Data
class ECG_CNN(nn.Module):
    def __init__(self):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3)
        self.pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(16 * 49, 128)  # Output 128-dim features

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x)

# Define Transformer Encoder Layer with Attention Gating for Tabular Data
class BidirectionalTransformer(nn.Module):
    def __init__(self, input_dim):
        super(BidirectionalTransformer, self).__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=4, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(self.transformer_layer, num_layers=2)
        self.fc = nn.Linear(input_dim, 128)

    def forward(self, x):
        x = self.transformer(x)
        return self.fc(x[:, 0, :])  # CLS token as summary

# BERT Model for Text Data
class ClinicalTextBERT(nn.Module):
    def __init__(self):
        super(ClinicalTextBERT, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 128)

    def forward(self, x):
        x = self.bert(x).last_hidden_state[:, 0, :]
        return self.fc(x)

# Cross-Attention Mechanism for Fusion
class CrossAttentionFusion(nn.Module):
    def __init__(self):
        super(CrossAttentionFusion, self).__init__()
        self.cross_attention = nn.MultiheadAttention(embed_dim=384, num_heads=4, batch_first=True)
        self.fc = nn.Linear(384, 128)

    def forward(self, ecg, tabular, text):
        # Concatenate along feature dimension to ensure 384-dim input to attention
        combined = torch.cat([ecg.unsqueeze(1), tabular.unsqueeze(1), text.unsqueeze(1)], dim=-1)
        combined = combined.permute(1, 0, 2)  # Format for attention (seq_len, batch, embed_dim)
        
        # Multihead Attention
        attention_output, _ = self.cross_attention(combined, combined, combined)
        
        # Flatten and pass through final fully connected layer
        attention_output = attention_output.mean(dim=0)  # (batch, embed_dim)
        return self.fc(attention_output)

# Complete Hybrid Model: CNN-BiTAM
class CNN_BiTAM(nn.Module):
    def __init__(self):
        super(CNN_BiTAM, self).__init__()
        self.ecg_cnn = ECG_CNN()
        self.tabular_transformer = BidirectionalTransformer(128)
        self.text_bert = ClinicalTextBERT()
        self.fusion_layer = CrossAttentionFusion()
        self.classifier = nn.Linear(128, 1)

    def forward(self, ecg, tabular, text):
        ecg_features = self.ecg_cnn(ecg)
        tabular_features = self.tabular_transformer(tabular)
        text_features = self.text_bert(text)
        
        fused_features = self.fusion_layer(ecg_features, tabular_features, text_features)
        return torch.sigmoid(self.classifier(fused_features))

# Sample Data Preparation
ecg_data = torch.rand(32, 1, 100)         # ECG data (batch_size, channels, length)
tabular_data = torch.rand(32, 10, 128)    # Tabular data (batch_size, seq_length, feature_dim)
text_input = ["Patient has a history of chest pain and hypertension."] * 32
encoded_text = tokenizer(text_input, return_tensors="pt", padding=True, truncation=True)
text_data = encoded_text['input_ids']

# Instantiate Model
model = CNN_BiTAM()

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Example
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(ecg_data, tabular_data, text_data)
    target = torch.rand(32, 1).round()  # Example target (random binary labels)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Model Output
print("Sample Prediction:", output.detach().numpy())


/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch [1/100], Loss: 0.6932
Epoch [2/100], Loss: 0.9888
Epoch [3/100], Loss: 0.6625
Epoch [4/100], Loss: 0.6909
Epoch [5/100], Loss: 0.7381
Epoch [6/100], Loss: 0.7441
Epoch [7/100], Loss: 0.6555
Epoch [8/100], Loss: 0.8668
Epoch [9/100], Loss: 0.7201
Epoch [10/100], Loss: 0.6865
Epoch [11/100], Loss: 0.6979
Epoch [12/100], Loss: 0.7167
Epoch [13/100], Loss: 0.7469
Epoch [14/100], Loss: 0.6868
Epoch [15/100], Loss: 0.6918
Epoch [16/100], Loss: 0.7036
Epoch [17/100], Loss: 0.6710
Epoch [18/100], Loss: 0.7021
Epoch [19/100], Loss: 0.7564
Epoch [20/100], Loss: 0.7781
Epoch [21/100], Loss: 0.7480
Epoch [22/100], Loss: 0.6932
Epoch [23/100], Loss: 0.6994
Epoch [24/100], Loss: 0.7573
Epoch [25/100], Loss: 0.6982
Epoch [26/100], Loss: 0.6981
Epoch [27/100], Loss: 0.7046
Epoch [28/100], Loss: 0.7080
Epoch [29/100], Loss: 0.6930
Epoch [30/100], Loss: 0.6720
Epoch [31/100], Loss: 0.7003
Epoch [32/100], Loss: 0.6986
Epoch [33/100], Loss: 0.7049
Epoch [34/100], Loss: 0.6930
Epoch [35/100], Loss: 0

In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# Tokenizer initialization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing: Standardize Tabular Data and Prepare Text
scaler = StandardScaler()

# Sample Data Preparation
ecg_data = torch.rand(128, 1, 100)  # ECG data (batch_size, channels, length)
tabular_data = scaler.fit_transform(torch.rand(128, 10, 128).reshape(-1, 128)).reshape(128, 10, 128)
tabular_data = torch.tensor(tabular_data, dtype=torch.float32)
text_input = ["Patient has a history of chest pain and hypertension."] * 128
encoded_text = tokenizer(text_input, return_tensors="pt", padding=True, truncation=True)
text_data = encoded_text['input_ids']

# Train-Test Split
ecg_train, ecg_val, tab_train, tab_val, text_train, text_val = train_test_split(
    ecg_data, tabular_data, text_data, test_size=0.2, random_state=42
)

# Define Enhanced CNN for ECG Data
# Define Enhanced CNN for ECG Data
class ECG_CNN(nn.Module):
    def __init__(self):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 5)
        self.conv2 = nn.Conv1d(32, 64, 3)
        self.pool = nn.MaxPool1d(2)
        
        # Dynamically determine the flattened input size
        self._to_linear = None
        self.init_flattened_size()
        self.fc = nn.Linear(self._to_linear, 128)

    def init_flattened_size(self):
        # Pass a sample input through the layers to determine the output size
        x = torch.rand(1, 1, 100)  # Sample input (batch_size, channels, length)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        self._to_linear = x.numel()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten dynamically based on calculated size
        return self.fc(x)


# Define Transformer with Extra Layers and Attention for Tabular Data
class BidirectionalTransformer(nn.Module):
    def __init__(self, input_dim):
        super(BidirectionalTransformer, self).__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=8, batch_first=True)
        self.transformer = nn.TransformerEncoder(self.transformer_layer, num_layers=3)
        self.fc = nn.Linear(input_dim, 128)

    def forward(self, x):
        x = self.transformer(x)
        return self.fc(x[:, 0, :])

# Text Processing with ClinicalTextBERT
class ClinicalTextBERT(nn.Module):
    def __init__(self):
        super(ClinicalTextBERT, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 128)

    def forward(self, x):
        x = self.bert(x).last_hidden_state[:, 0, :]
        return self.fc(x)

# Cross-Attention Fusion Mechanism with More Layers
class CrossAttentionFusion(nn.Module):
    def __init__(self):
        super(CrossAttentionFusion, self).__init__()
        self.cross_attention = nn.MultiheadAttention(embed_dim=384, num_heads=8, batch_first=True)
        self.fc = nn.Linear(384, 128)
        self.dropout = nn.Dropout(0.3)

    def forward(self, ecg, tabular, text):
        combined = torch.cat([ecg.unsqueeze(1), tabular.unsqueeze(1), text.unsqueeze(1)], dim=-1)
        combined = combined.permute(1, 0, 2)
        
        attention_output, _ = self.cross_attention(combined, combined, combined)
        attention_output = attention_output.mean(dim=0)
        return self.dropout(self.fc(attention_output))

# Updated Hybrid Model with Enhanced Components
class CNN_BiTAM(nn.Module):
    def __init__(self):
        super(CNN_BiTAM, self).__init__()
        self.ecg_cnn = ECG_CNN()
        self.tabular_transformer = BidirectionalTransformer(128)
        self.text_bert = ClinicalTextBERT()
        self.fusion_layer = CrossAttentionFusion()
        self.classifier = nn.Linear(128, 1)

    def forward(self, ecg, tabular, text):
        ecg_features = self.ecg_cnn(ecg)
        tabular_features = self.tabular_transformer(tabular)
        text_features = self.text_bert(text)
        
        fused_features = self.fusion_layer(ecg_features, tabular_features, text_features)
        return torch.sigmoid(self.classifier(fused_features))

# Model Initialization
model = CNN_BiTAM()

# Define weighted loss for class imbalance and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

# Ensure consistent batch size
batch_size = 128
target = torch.rand(batch_size, 1).round()  # Example target (random binary labels)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to calculate metrics
def calculate_metrics(outputs, targets):
    outputs = (outputs > 0.5).float()  # Convert probabilities to binary predictions
    targets = targets.float()
    accuracy = accuracy_score(targets.cpu(), outputs.cpu())
    precision = precision_score(targets.cpu(), outputs.cpu())
    recall = recall_score(targets.cpu(), outputs.cpu())
    f1 = f1_score(targets.cpu(), outputs.cpu())
    return accuracy, precision, recall, f1

# Training Loop with Metrics Calculation
epochs = 100
batch_size = 128
target = torch.rand(batch_size, 1).round()  # Example target (random binary labels)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Training Batch
    ecg_batch = ecg_train[:batch_size]
    tab_batch = tab_train[:batch_size]
    text_batch = text_train[:batch_size]
    
    output = model(ecg_batch, tab_batch, text_batch)
    
    # Ensure target size matches output
    target = target[:output.size(0)]
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        ecg_val_batch = ecg_val[:batch_size]
        tab_val_batch = tab_val[:batch_size]
        text_val_batch = text_val[:batch_size]

        val_output = model(ecg_val_batch, tab_val_batch, text_val_batch)
        val_target = target[:val_output.size(0)]
        val_loss = criterion(val_output, val_target)
        scheduler.step(val_loss)
        
        # Calculate metrics
        accuracy, precision, recall, f1 = calculate_metrics(val_output, val_target)

    print(f"Epoch [{epoch + 1}/{epochs}], "
          f"Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
          f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
          f"Recall: {recall:.4f}, F1 Score: {f1:.4f}")

print("Training completed. Evaluating final model performance...")


/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/100], Loss: 0.6867, Val Loss: 0.6942, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [2/100], Loss: 0.7377, Val Loss: 0.7913, Accuracy: 0.3462, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [3/100], Loss: 0.7727, Val Loss: 0.7126, Accuracy: 0.3462, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [4/100], Loss: 0.7018, Val Loss: 0.7151, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [5/100], Loss: 0.7747, Val Loss: 0.6576, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [6/100], Loss: 0.6708, Val Loss: 0.6602, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [7/100], Loss: 0.6733, Val Loss: 0.6454, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [8/100], Loss: 0.6639, Val Loss: 0.6795, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [9/100], Loss: 0.6815, Val Loss: 0.6451, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [10/100], Loss: 0.6690, Val Loss: 0.6500, Accuracy: 0.6538, Precision: 0.6538, Recall: 1.0000, F1 Score: 0.7907
Epoch [11/100], Loss: 0.6637, Val Loss: 0.6457, Accuracy: 0.653

In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Tokenizer initialization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing: Standardize Tabular Data and Prepare Text
scaler = StandardScaler()

# Sample Data Preparation
ecg_data = torch.rand(500, 1, 100)  # Larger dataset (batch_size, channels, length)
tabular_data = scaler.fit_transform(torch.rand(500, 10, 128).reshape(-1, 128)).reshape(500, 10, 128)
tabular_data = torch.tensor(tabular_data, dtype=torch.float32)
text_input = ["Patient has a history of chest pain and hypertension."] * 500
encoded_text = tokenizer(text_input, return_tensors="pt", padding=True, truncation=True)
text_data = encoded_text['input_ids']

# Train-Test Split
ecg_train, ecg_val, tab_train, tab_val, text_train, text_val = train_test_split(
    ecg_data, tabular_data, text_data, test_size=0.2, random_state=42
)

# Define Enhanced CNN for ECG Data
class ECG_CNN(nn.Module):
    def __init__(self):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 5)
        self.conv2 = nn.Conv1d(32, 64, 3)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        self._to_linear = None
        self.init_flattened_size()
        self.fc = nn.Linear(self._to_linear, 128)
        self.bn = nn.BatchNorm1d(128)

    def init_flattened_size(self):
        x = torch.rand(1, 1, 100)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        self._to_linear = x.numel()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.bn(self.fc(x))
        return self.dropout(x)

# Define Transformer with Extra Layers and Attention for Tabular Data
class BidirectionalTransformer(nn.Module):
    def __init__(self, input_dim):
        super(BidirectionalTransformer, self).__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=8, batch_first=True)
        self.transformer = nn.TransformerEncoder(self.transformer_layer, num_layers=3)
        self.fc = nn.Linear(input_dim, 128)
        self.bn = nn.BatchNorm1d(128)

    def forward(self, x):
        x = self.transformer(x)
        return self.bn(self.fc(x[:, 0, :]))

# Text Processing with ClinicalTextBERT
class ClinicalTextBERT(nn.Module):
    def __init__(self):
        super(ClinicalTextBERT, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, 128)
        self.bn = nn.BatchNorm1d(128)

    def forward(self, x):
        x = self.bert(x).last_hidden_state[:, 0, :]
        return self.bn(self.fc(x))

# Cross-Attention Fusion Mechanism with More Layers
class CrossAttentionFusion(nn.Module):
    def __init__(self):
        super(CrossAttentionFusion, self).__init__()
        self.cross_attention = nn.MultiheadAttention(embed_dim=384, num_heads=8, batch_first=True)
        self.fc = nn.Linear(384, 128)
        self.dropout = nn.Dropout(0.5)

    def forward(self, ecg, tabular, text):
        combined = torch.cat([ecg.unsqueeze(1), tabular.unsqueeze(1), text.unsqueeze(1)], dim=-1)
        combined = combined.permute(1, 0, 2)
        attention_output, _ = self.cross_attention(combined, combined, combined)
        attention_output = attention_output.mean(dim=0)
        return self.dropout(self.fc(attention_output))

# Updated Hybrid Model with Enhanced Components
class CNN_BiTAM(nn.Module):
    def __init__(self):
        super(CNN_BiTAM, self).__init__()
        self.ecg_cnn = ECG_CNN()
        self.tabular_transformer = BidirectionalTransformer(128)
        self.text_bert = ClinicalTextBERT()
        self.fusion_layer = CrossAttentionFusion()
        self.classifier = nn.Linear(128, 1)

    def forward(self, ecg, tabular, text):
        ecg_features = self.ecg_cnn(ecg)
        tabular_features = self.tabular_transformer(tabular)
        text_features = self.text_bert(text)
        fused_features = self.fusion_layer(ecg_features, tabular_features, text_features)
        return torch.sigmoid(self.classifier(fused_features))

# Model Initialization
model = CNN_BiTAM()

# Define weighted loss for class imbalance and optimizer
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Training configurations
batch_size = 32
epochs = 100
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Function to calculate metrics
def calculate_metrics(outputs, targets):
    outputs = (outputs > 0.5).float()
    targets = targets.float()
    accuracy = accuracy_score(targets.cpu(), outputs.cpu())
    precision = precision_score(targets.cpu(), outputs.cpu())
    recall = recall_score(targets.cpu(), outputs.cpu())
    f1 = f1_score(targets.cpu(), outputs.cpu())
    return accuracy, precision, recall, f1

# Training Loop with Early Stopping and Metrics Calculation
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Mini-batch training
    for i in range(0, len(ecg_train), batch_size):
        ecg_batch = ecg_train[i:i+batch_size]
        tab_batch = tab_train[i:i+batch_size]
        text_batch = text_train[i:i+batch_size]
        
        output = model(ecg_batch, tab_batch, text_batch)
        batch_target = torch.rand(output.size(0), 1).round()  # Simulated target
        loss = criterion(output, batch_target)
        
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(ecg_val, tab_val, text_val)
        val_target = torch.rand(val_output.size(0), 1).round()
        val_loss = criterion(val_output, val_target)
        scheduler.step(epoch + val_loss)  # Adjust LR dynamically with epoch + val_loss

        # Calculate metrics
        accuracy, precision, recall, f1 = calculate_metrics(val_output, val_target)

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    print(f"Epoch [{epoch + 1}/{epochs}], "
          f"Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
          f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
          f"Recall: {recall:.4f}, F1 Score: {f1:.4f}")

print("Training completed. Evaluating final model performance...")


/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [1/100], Loss: 0.6780, Val Loss: 0.7290, Accuracy: 0.5300, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [2/100], Loss: 0.7906, Val Loss: 0.7135, Accuracy: 0.5200, Precision: 0.5200, Recall: 1.0000, F1 Score: 0.6842
Epoch [3/100], Loss: 0.6767, Val Loss: 0.7054, Accuracy: 0.4800, Precision: 0.4800, Recall: 1.0000, F1 Score: 0.6486
Epoch [4/100], Loss: 0.7062, Val Loss: 0.8634, Accuracy: 0.5100, Precision: 0.5100, Recall: 1.0000, F1 Score: 0.6755


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [5/100], Loss: 0.7204, Val Loss: 1.2983, Accuracy: 0.5900, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch [14/100], Loss: 0.7983, Val Loss: 0.8558, Accuracy: 0.5200, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [15/100], Loss: 0.6963, Val Loss: 0.7735, Accuracy: 0.5100, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [16/100], Loss: 0.6314, Val Loss: 0.7957, Accuracy: 0.4400, Precision: 0.4400, Recall: 1.0000, F1 Score: 0.6111


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [17/100], Loss: 0.7336, Val Loss: 0.7703, Accuracy: 0.4800, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [18/100], Loss: 0.6667, Val Loss: 0.9534, Accuracy: 0.5000, Precision: 0.5000, Recall: 1.0000, F1 Score: 0.6667
Epoch [19/100], Loss: 0.7377, Val Loss: 0.7076, Accuracy: 0.4800, Precision: 0.4800, Recall: 1.0000, F1 Score: 0.6486
Epoch [20/100], Loss: 0.7196, Val Loss: 0.6692, Accuracy: 0.6100, Precision: 0.6100, Recall: 1.0000, F1 Score: 0.7578
Epoch [21/100], Loss: 0.7666, Val Loss: 0.7684, Accuracy: 0.5000, Precision: 0.5000, Recall: 1.0000, F1 Score: 0.6667
Epoch [22/100], Loss: 0.6856, Val Loss: 0.6903, Accuracy: 0.5400, Precision: 0.5422, Recall: 0.8491, F1 Score: 0.6618
Epoch [23/100], Loss: 0.6676, Val Loss: 0.6867, Accuracy: 0.5400, Precision: 0.4286, Recall: 0.0667, F1 Score: 0.1154
Epoch [24/100], Loss: 0.7182, Val Loss: 0.6921, Accuracy: 0.5400, Precision: 0.5476, Recall: 0.8519, F1 Score: 0.6667
Epoch [25/100], Loss: 0.6760, Val Loss: 0.6951, Accuracy

In [68]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess the Parkinson's dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data'
df = pd.read_csv(url)

# Features and target
X = df.drop(['name', 'status'], axis=1)
y = df['status'].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert data to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TransformerModel, self).__init__()
        self.fc_in = nn.Linear(input_dim, 64)
        encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc_out = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.dropout(x)
        x = x.unsqueeze(1)  # Add sequence dimension
        x = self.transformer_encoder(x)
        x = x.squeeze(1)    # Remove sequence dimension
        x = self.fc_out(x)
        return x

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
epochs = 100
batch_size = 16
patience = 10

all_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(X_tensor)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_tensor[train_index], X_tensor[val_index]
    y_train, y_val = y_tensor[train_index], y_tensor[val_index]

    # Create DataLoaders
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, criterion, optimizer, scheduler
    input_dim = X_train.shape[1]
    num_classes = len(torch.unique(y_tensor))
    model = TransformerModel(input_dim, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

    # Early stopping parameters
    best_accuracy = 0
    trigger_times = 0

    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Update learning rate
        scheduler.step()

        # Validation
        model.eval()
        y_val_pred = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                y_val_pred.extend(predicted.numpy())
        val_accuracy = accuracy_score(y_val.numpy(), y_val_pred)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            trigger_times = 0
            torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy * 100:.2f}%")

    print(f"Best Validation Accuracy for Fold {fold + 1}: {best_accuracy * 100:.2f}%")
    all_accuracies.append(best_accuracy)

# Calculate average accuracy across folds
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Cross-Validation Accuracy: {average_accuracy * 100:.2f}%")



Fold 1


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10/100, Validation Accuracy: 87.18%
Early stopping!
Best Validation Accuracy for Fold 1: 87.18%

Fold 2


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10/100, Validation Accuracy: 82.05%
Epoch 20/100, Validation Accuracy: 89.74%
Epoch 30/100, Validation Accuracy: 94.87%
Early stopping!
Best Validation Accuracy for Fold 2: 94.87%

Fold 3


...

Epoch 10/100, Validation Accuracy: 89.74%
Early stopping!
Best Validation Accuracy for Fold 3: 94.87%

Fold 4


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10/100, Validation Accuracy: 84.62%
Early stopping!
Best Validation Accuracy for Fold 4: 87.18%

Fold 5


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10/100, Validation Accuracy: 87.18%
Early stopping!
Best Validation Accuracy for Fold 5: 87.18%

Average Cross-Validation Accuracy: 90.26%
